## Visualization of model performance

In [ ]:
import pathlib
import random
import torch
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset

from models import ConvRegv2
from mlcore.training import make_predictions
from mlcore.eval import plot_stream_data
from mlcore.dataset import load_training_data, stream_to_arrival, stream_to_height

#### Model Loading

In [ ]:
# Load model
MODEL_DIR = pathlib.Path().cwd() / 'trained_models'
MODEL_FNAME = 'cnn_reg_1691256621.pt'
RANDOM_SEED = 42
TEST_RATIO = 0.2
BATCH_SIZE = 32
EDGE_PAD = 10
WINDOW_SIZE = 1000
NUM_SAMPLES = 1000

if torch.cuda.is_available():
  device = torch.device("cuda")

elif torch.backends.mps.is_available():
  device = torch.device('mps')

else:
  device = torch.device("cpu")
print(f'Using device: "{device}"')

model = ConvRegv2(2, BATCH_SIZE)
model.load_state_dict(torch.load(MODEL_DIR / MODEL_FNAME, map_location=device))

### Data Loading

In [ ]:
# Define dataset locations and load the training and test data
test_dir = Path('../../../data/pulses/test/single_pulse/variable_qp_density/raw_iq')
train_dir = Path('../../../data/pulses/train/single_pulse/variable_qp_density/raw_iq')
fname = Path(f'vp_single_num{NUM_SAMPLES}_win{WINDOW_SIZE}_pad{EDGE_PAD}.npz')
labels = ('i', 'q', 'photon_arrivals', 'phase_response')
i_test, q_test, arrs_test, theta1_test = load_training_data(test_dir / fname, labels=labels)
i_train, q_train, arrs_train, theta1_train = load_training_data(train_dir / fname, labels=labels)

### Data Transformations

In [ ]:
# Now we want to expand the dimensions for the i and q streams
# since they will be used as input samples.
i_test = np.expand_dims(i_test, axis=1)
i_train = np.expand_dims(i_train, axis=1)
q_test = np.expand_dims(q_test, axis=1)
q_train = np.expand_dims(q_train, axis=1)

# Get pulse heights and photon arrival values
target_arrs_train = stream_to_arrival(arrs_train)
target_arrs_test = stream_to_arrival(arrs_test)
target_pulse_train = stream_to_height(arrs_train, theta1_train)
target_pulse_test = stream_to_height(arrs_test, theta1_test)

# Now we want to convert the loaded data to tensors.
# Shape for targets is NUM_SAMPLES x 1 x 2
# Shape for inputs is NUM_SAMPLES x 2 x WINDOW_SIZE
X_train = torch.Tensor(np.hstack((i_train, q_train))) 
X_test = torch.Tensor(np.hstack((i_test, q_test)))
y_train = torch.Tensor(np.stack((target_arrs_train, target_pulse_train), axis=2)) 
y_test = torch.Tensor(np.stack((target_arrs_test, target_pulse_test), axis=2))

# From the newly created tensors, create testing and training datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

### Model Predictions and Visualization

In [ ]:
# Pick k random samples/labels from the test data and plot them along with the predictions
test_samples = []
test_labels = []

for sample, label in random.sample(list(test_dataset), k=len(test_dataset)): # random.sample samples k elements from the given population without replacement; returns list of samples.
    test_samples.append(sample)
    test_labels.append(label)

print(f'Test Sample Shape: {test_samples[0].shape}, Test Label Shape: {test_labels[0].shape}')
preds = make_predictions(model, [x.unsqueeze(dim=0) for x in test_samples]) # returns a tensor
print(f'Preds shape {preds[0].shape}')

In [ ]:
# Since the train/test loss was so low and the absolute error was low, lets determine the difference between the predicted and
# target arival times and plot this
# Note the multiplication by 1000 to get back to the arrival time element
arrival_diff = [torch.abs(WINDOW_SIZE * y_pred[0][0][0] - WINDOW_SIZE * y_true[0][0]).item() for y_pred, y_true in zip(preds, test_labels)]

plt.figure()
plt.title('Photon Arrival Time Error')
plt.plot(np.arange(len(arrival_diff)), arrival_diff)
plt.xlabel('Test Sample')
plt.ylabel('Error (samples)')
plt.show()

In [ ]:
# Now that the error is known, let's plot the actual values for both the predictions and the targets
plt.figure(figsize=(12,7))
plt.title('Photon Arrival Time Predicted vs Target')
#plt.plot(np.arange(len(preds)), [WINDOW_SIZE*pred[0][0][0].item() for pred in preds], label='Predicted')
#plt.plot(np.arange(len(test_labels)), [WINDOW_SIZE*label[0][0].item() for label in test_labels], label='Target')
plt.scatter(np.arange(len(preds)), [WINDOW_SIZE*pred[0][0][0].item() for pred in preds], marker='+', label='Predicted')
plt.scatter(np.arange(len(test_labels)), [WINDOW_SIZE*label[0][0].item() for label in test_labels], label='Target')
plt.xlabel('Test Sample')
plt.ylabel('Photon Arrival (sample)')
plt.legend()
plt.show()

In [ ]:
# Now lets do the same with the pulse height
height_diff = [torch.abs(y_pred[0][0][1] - y_true[0][1]).item() for y_pred, y_true in zip(preds, test_labels)]

plt.figure()
plt.title('Error in Pulse Height')
plt.plot(np.arange(len(height_diff)), height_diff)
plt.xlabel('Test Sample')
plt.ylabel('Error')
plt.show()

plt.figure(figsize=(12,7))
plt.title('Pulse Height Predicted vs Target')
#plt.plot(np.arange(len(preds)), [pred[0][0][1].item() for pred in preds], label='Predicted')
#plt.plot(np.arange(len(test_labels)), [label[0][1].item() for label in test_labels], label='Target')
plt.scatter(np.arange(len(preds)), [pred[0][0][1].item() for pred in preds], label='Predicted')
plt.scatter(np.arange(len(test_labels)), [label[0][1].item() for label in test_labels], marker='+', label='Target')
plt.xlabel('Test Sample')
plt.ylabel('Pulse Height (a.u.)')
plt.legend()
plt.show()

print(f'Mean height error: {np.mean(height_diff)}, Mean height error ratio: {np.mean(height_diff)/np.mean([target[0][1] * -1 for target in test_labels])}')
print(f'median height error: {np.median(height_diff)}, median height error ratio: {np.median(height_diff)/np.median([target[0][1] * -1 for target in test_labels])}')

In [ ]:
for target, pred in zip(np.array(test_labels)[20:30], preds[20:30]):
   plt.figure()
   pred_phase_resp = pred[0][0][1].item()
   target_phase_resp = target[0][1].item()
   tgt, pred = np.ones(11), np.ones(11)
   tgt[5] = target_phase_resp
   pred[5] = pred_phase_resp
   plt.plot(np.arange(11), tgt, label='target')
   plt.plot(np.arange(11), pred, label='pred')
   plt.legend()
   plt.show()
   print(f'Diff: {np.abs(target_phase_resp - pred_phase_resp)}')


#### The model seems to be doing very well predicting the photon arrival time (looks like average error is ~5us) but not so well predicting the non-normalized phase response peaks. This seems to make sense because the phase response is affected by noise whereas the previous models were trained only on a set number of possible pulse heights. Since we aren't tied to the phase response transformation that the optimal filter is, I think a good step would be to try and normalize the phase response pulses. The loss also seems to be concentrated in the arrival time space, meaning it may be a good idea to separate the two from a loss function perspective (or maybe even creating a separate FC network just for phase response).

### Energy Prediction and Resolution

#### Let's plot a histogram of the quasiparticle perturbation predictions from the model

In [ ]:
# Create the histogram of all values and then plot
qp_preds = np.array([pred[0][0][1] for pred in preds]) * -1
counts, bins = np.histogram(qp_preds, range=(0.1, qp_preds.max() + (0.2 * qp_preds.max())), bins=200)

# Recover true pulse heights from the test label data
qp_true = np.unique([target.squeeze()[1].item() for target in test_labels])

# Lets plot the histogram
plt.figure(figsize=(12,7))
plt.title('Predicted Pulse Heights')
plt.stairs(counts, bins)

# Need to add in the true pulse heights
for target in qp_true:
    pass
    #plt.axvline(target * -1, c='r')
plt.xlabel('Pulse Height')
plt.ylabel('Counts')
plt.show()

In [ ]:
# Create the histogram of all values and then plot
targets = np.array([target.squeeze()[1].item() for target in test_labels]) * -1
counts, bins = np.histogram(targets, range=(0.1, targets.max() + (0.2 * targets.max())), bins=200)

# Lets plot the histogram
plt.figure(figsize=(12,7))
plt.title('Target Pulse Heights (Normalized)')
plt.stairs(counts, bins)
plt.xlabel('Pulse Height')
plt.ylabel('Counts')
plt.show()

In [ ]:
targets = stream_to_height(arrs_test, theta1_test, norm=False)
counts, bins = np.histogram(targets, range=(targets.min() + (0.2 * targets.min()), 0.1), bins=50)

# Lets plot the histogram
plt.figure(figsize=(12,7))
plt.title('Target Pulse Heights (Raw)')
plt.stairs(counts, bins)
plt.xlabel('Pulse Height')
plt.ylabel('Counts')
plt.show()